# 🦜🔗 LangChain - Agents & Tools

© Advanced Analytics, Amir Ben Haim, 2024

<br>
<br>
<hr class="dotted">
<br>
<br>

## Agents & Tools

- **Agents**
  - Agents - are advanced LLM applications that can decide what steps to take, in what order, **using "tools" to accomplish complex tasks**
  - Instead of static chains, agents can reason, select tools, and loop until a goal is met — enabling more dynamic, autonomous workflows
  - Agent uses tool descriptions to decide when/how to call

- **Tools**
  - Tools - are Python functions, APIs, or integrations that an agent can use to fetch data, calculate, search, etc
  - A <u>toolkit</u> is a bundle of related tools (e.g., file search, web browsing, code execution)
  - Tools must be **structured functions** or **OpenAI function calling style**
  - LangChain provides many built-in tools (math, search, Python REPL, and more), but you can define your own

<br>
<br>
<hr class="dotted">
<br>
<br>

## Agents vs. Chains

| Chains                     | Agents                                    |
|----------------------------|-------------------------------------------|
| Fixed sequence of steps    | Dynamic reasoning (choose next action)    |
| No decision-making         | Makes decisions at each step              |
| Good for linear workflows  | Good for uncertain/multi-step problems    |
| Example: Translate, then summarize | Example: Lookup docs, search web, use calculator |

<br>
<br>
<hr class="dotted">
<br>
<br>

## Setup

<br></br>

### <u>API Keys</u>

In order to use the OpenAI language model, users are required to generate a token.
<br></br>
<u>Follow these simple steps to generate a token with openai:</u>
- Go to <a href="url">https://platform.openai.com/apps</a>  and signup with your email address or connect your Google Account.
- Go to View API Keys on left side of your Personal Account Settings
- Select Create new Secret key
- The API access to OPENAI is a paid service
- You have to set up billing
- You don’t need ChatGPT Plus - The API and ChatGPT subscriptions are billed separately
<br></br>
<p style="background-color:Tomato"> Make sure you read the Pricing information before experimenting</p>
<p style="background-color:Tomato">Once you add your API key, make sure to not share it with anyone! The API key should remain private</p>
<p style="background-color:Tomato">Use the <code>.env</code> file for you API key</p>

<br></br>

### <u>pip install</u>

```powershell
pip install langchain langchain-openai langchain-community
pip install python-dotenv
```

<br></br>

### <u>API Key Setup</u>

Before using LangChain with OpenAI, set your API key:

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads variables from .env

openai_key = os.getenv("OPENAI_API_KEY")
print(openai_key[:5])  # Just to check, don't print the full key!

<br>
<br>
<hr class="dotted">
<br>
<br>

## Example 1: Agent With Calculator Tool

<br></br>

### <u>Define a Simple Calculator Tool</u>

In [ ]:
from langchain.tools import tool

In [ ]:
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers and return the result."""
    return a + b

`@tool`
<br>

- It's a **decorator**

- A convenient way to convert a standard Python function into a tool that can be utilized by language models (LLMs) within agent workflows

- LangChain automatically:
  - **Assigns a name**: By default, it uses the function's name, but you can specify a custom name
  - **Generates a description**: It uses the function's docstring to describe the tool's purpose
  - **Infers the input schema**: It analyzes the function's parameters and type annotations to determine the expected inputs

- In this example:
  - **Name**: add_numbers
  - **Description**: "Add two numbers and return the result."
  - **Arguments**: a and b, both integers
  - The LLM can now call add_numbers as a tool when it determines that adding two numbers is necessary to fulfill a user's request

<br><br>

<u>**Pay Attention to the Error**</u>

<pre> ```ValueError: Function must have a docstring if description not provided. ``` </pre>

In [ ]:
#Error


@tool
def add_numbers(a: int, b: int) -> int:
    return a + b

<br></br>

### <u>Build an Agent With the Tool</u>

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

<br></br>

<u>**PAY ATTENTION !!!**</u>
<br>
The order in the **PROMPT TEMPLATE** matter:
- `system`
- `MessagesPlaceholder(variable_name="agent_scratchpad")`
- `user`

In [ ]:
# Define the tools
tools = [add_numbers]



# Create prompt for the agent (optional for custom role)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can use tools when needed."),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("user", "{input}")
])



llm = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0)



# Build agent
agent = create_openai_tools_agent(llm, tools, prompt)



# Create an executor (to run the agent)
agent_executor = AgentExecutor(agent=agent, tools=tools)



# print
response = agent_executor.invoke({"input": "What is 7 plus 15?"})
print(response["output"])


In [ ]:
response

In [ ]:
type(response)

<br></br>

### <u>Adding **verbose=True**</u>

`agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)`
<br>

- Enables detailed logging of the agent's internal operations
- This includes printing intermediate steps, such as the agent's thoughts, actions, and observations, to the console
- It's particularly useful for debugging and understanding how the agent processes inputs and utilizes tools
- **You can inspect the reasoning/steps by setting `verbose=True` on the executor**

In [ ]:
# Define the tools
tools = [add_numbers]



# Create prompt for the agent (optional for custom role)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can use tools when needed."),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("user", "{input}")
])



llm = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0)



# Build agent
agent = create_openai_tools_agent(llm, tools, prompt)



# Create an executor (to run the agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



# print
response = agent_executor.invoke({"input": "What is 7 plus 15?"})

print('\n\n')
print('Agent Response')
print(len('Agent Response')*'-')
print(response["output"])


<br>
<br>
<hr class="dotted">
<br>
<br>

## Example 2: Multi-Tool Agent (Calculator & Search)

<br></br>

### <u>Define a Second Tool</u>

In [ ]:
@tool
def get_country_capital(country: str) -> str:
    """Return the capital city of a given country."""
    capitals = {
        "France": "Paris",
        "Israel": "Jerusalem",
        "USA": "Washington, D.C."
    }
    return capitals.get(country, "Unknown")

<br></br>

### <u>Agent With Multiple Tools</u>

In [ ]:
# Define the tools
tools = [add_numbers, get_country_capital]



# Create prompt for the agent (optional for custom role)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can use tools when needed."),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("user", "{input}")
])



llm = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0)



# Build agent
agent = create_openai_tools_agent(llm, tools, prompt)



# Create an executor (to run the agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



# Ask it a question that could use a tool
response = agent_executor.invoke({"input": "What is the capital of France and what is 3 + 4?"})

print('\n\n')
print('Agent Response')
print(len('Agent Response')*'-')
print(response["output"])

<br></br>

### <u>Agent With Multiple Tools - with a query that <b>doesn't require the tools</b></u>

In [ ]:
# Define the tools
tools = [add_numbers, get_country_capital]



# Create prompt for the agent (optional for custom role)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can use tools when needed."),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("user", "{input}")
])



llm = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0)



# Build agent
agent = create_openai_tools_agent(llm, tools, prompt)



# Create an executor (to run the agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



# Ask it a question that could use a tool
response = agent_executor.invoke({"input": "Tell a short joke"})

print('\n\n')
print('Agent Response')
print(len('Agent Response')*'-')
print(response["output"])

<br></br>

### <u>Agent Flow</b></u>

- Agents decide when to call a tool vs. answer directly

- If user asks “what is 3 + 4?” → agent will use the calculator tool

- If user asks “Tell me a joke” → agent will just use the LLM

<br>
<br>
<hr class="dotted">
<br>
<br>

## Example 3: Agent Chaining: Using Outputs From One Tool In Another

<br></br>

### <u>Define the Tools</u>

In [ ]:
@tool
def multiply(a: int) -> int:
    """Multiply a number by 2."""
    return a * 2

@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

<br></br>

### <u>Agent With Multiple Tools</u>

In [ ]:
# Define the tools
tools = [multiply, add]



# Create prompt for the agent (optional for custom role)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can use tools when needed."),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("user", "{input}")
])



llm = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0)



# Build agent
agent = create_openai_tools_agent(llm, tools, prompt)



# Create an executor (to run the agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



# Ask it a question that could use a tool
response = agent_executor.invoke({"input": "If I add 3 and 4, then double it, what do I get?"})

print('\n\n')
print('Agent Response')
print(len('Agent Response')*'-')
print(response["output"])

<br>
<br>
<hr class="dotted">
<br>
<br>

## Example 4: Agent With “Session” Memory

In [ ]:
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

<br></br>

### <u>Define the Tools</u>

In [ ]:
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers and return the result."""
    return a + b

<br></br>

### <u>Agent With Memory</u>

<br></br>

<u>**PAY ATTENTION !!!**</u>
<br>
The order in the **PROMPT TEMPLATE** matter:
- `system`
- `MessagesPlaceholder(variable_name="history")`
- `MessagesPlaceholder(variable_name="agent_scratchpad")`
- `user`

In [ ]:
# Define the tools
tools = [add_numbers]




# Create prompt for the agent (optional for custom role)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant who remembers prior conversation."),
    MessagesPlaceholder(variable_name="history"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("user", "{input}")
])




llm = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0)




# Build agent
agent = create_openai_tools_agent(llm, tools, prompt)




# Create an executor (to run the agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)




# Wrap with message history for memory
store = {}

def get_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]



chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history"
)




session_id = "student"
chat_agent.invoke({"input": "Add 5 and 7"}, config={"configurable": {"session_id": session_id}})
output = chat_agent.invoke({"input": "What was my last question?"}, config={"configurable": {"session_id": session_id}})


print('\n\n')
print('Agent Response')
print(len('Agent Response')*'-')
print(output["output"])

In [ ]:
store['student'].messages

<br>
<br>
<hr class="dotted">
<br>
<br>

## Example 5: Multi-Agent Orchestration

- Multi-agent orchestration is an advanced workflow design pattern where several specialized agents—each with their own prompt, role, tools, and potentially memory—collaborate to solve complex tasks
- Instead of relying on a single all-knowing agent, you coordinate several agents that can communicate, delegate, and share information.

<br></br>

### <u>Goal</u>

- **Have a "main agent" handle user conversation**
- If the user asks to "schedule an appointment," the main agent sends the full conversation to an "advisor agent"
- The advisor extracts the desired date, then suggests 3 appointment options
- The advisor will use a custom function for the 3 suggested appointment options
- **In this example only the "main agent" has memory**

In [ ]:
from datetime import datetime, timedelta

#Creating the function
@tool
def get_next_three_dates(start_date):
    'This fuction recieves a date and then return 3 optional dates'
    # start_date should be a string in the format 'YYYY-MM-DD'
    date_obj = datetime.strptime(start_date, "%Y-%m-%d")
    return [
        (date_obj + timedelta(days=3)).strftime("%Y-%m-%d"),
        (date_obj + timedelta(days=6)).strftime("%Y-%m-%d"),
        (date_obj + timedelta(days=9)).strftime("%Y-%m-%d"),
    ]

# Example usage:
# print(get_next_three_dates('2024-05-27'))

In [ ]:
# Define the tools
tools = [get_next_three_dates]


llm = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0)




# Main agent: Handles chat and decides when to call advisor
main_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an assistant in a company. If the user wants to schedule an appointment, respond: "
     "'I will check available slots for you.' Otherwise, answer normally."),
     MessagesPlaceholder(variable_name="history"),
     MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("user", "{input}")
])


main_agent = create_openai_tools_agent(llm, tools=[], prompt=main_prompt)
main_executor = AgentExecutor(agent=main_agent, tools=[], verbose=False)



# Memory store for user sessions
store = {}
def get_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Wrap the main agent executor with memory
main_agent_with_memory = RunnableWithMessageHistory(
    main_executor,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history"
)





# Advisor agent: Receives the conversation, extracts the desired date, and suggests slots
advisor_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an appointment advisor. Extract any preferred dates from the full conversation. "
     "Then, suggest 3 possible appointment slots for the user, in this format:\n"
     "- Option 1: ...\n- Option 2: ...\n- Option 3: ...\n"
     "You can use the tools provided"
     "If no date is found, suggest generic options for next week."),
     MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("user", "{input}")
])

advisor_agent = create_openai_tools_agent(llm, tools, prompt=advisor_prompt)
advisor_executor = AgentExecutor(agent=advisor_agent, tools=tools, verbose=True)

<u>**Orchestration**:</u>
- A Python-driven control layer that determines when and how agents interact with each other and with the user.

In [ ]:
def orchestrate_conversation_with_memory(user_input, session_id="user1"):
    """
    Handles one turn of user input for the main agent (with memory),
    and if needed, passes the full memory/history to the advisor agent.
    """


    # Main agent receives latest user message (memory auto-injects context)
    main_output = main_agent_with_memory.invoke({"input": user_input},config={"configurable": {"session_id": session_id}})["output"]
    print("Main Agent:", main_output)
    print("\n")

    # If scheduling is detected, advisor gets *full conversation* from memory
    if "I will check available slots for you" in main_output:
        
        # Retrieve full history from memory
        full_history = store[session_id].messages
        full_convo = "\n".join([f"{m.type.capitalize()}: {m.content}" for m in full_history])
        
        advisor_response = advisor_executor.invoke({"input": full_convo})["output"]
        print("\n\n")
        print("Advisor Agent:\n", advisor_response)

In [ ]:
session_id = "user77"
# Turn 1
orchestrate_conversation_with_memory("Hi! I want to learn about your company.", session_id=session_id)

In [ ]:
session_id = "user77"
# Turn 2
orchestrate_conversation_with_memory("I'd like to schedule an appointment on 2024-09-02.", session_id=session_id)
# ...keep calling per turn as needed...

<br></br>

<u>**Benefit of using Multi-Agent Orchestration**</u>

- **Separation of Concerns**: Each agent can focus on its domain (e.g., scheduling, knowledge retrieval, sales)

- **Composability**: Easily add, remove, or swap agents without redesigning the whole workflow

- **Dynamic Collaboration**: Agents can pass information, escalate issues, or request advice from other agents

- **Scalability**: Break down large business processes into modular, maintainable components

<br>
<br>
<hr class="dotted">
<br>
<br>

## Summary

- All custom tools must be decorated with `@tool` and include docstrings for best results

- Agents support function calling, so the LLM knows how/when to use each tool

- **For production, limit which tools the agent can access and sanitize tool inputs**

- Agents + memory enable stateful, contextual workflows

- Multi-agent orchestration lets you build robust, scalable, and maintainable LLM-powered systems where agents collaborate like human teams—each focusing on what they do best